In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_columns', None)   
pd.set_option('display.width', None)         

abandoned_release_history = pd.read_csv('/Users/kaziamithasan/Desktop/Research/msr25/msr25-mining-challenge/phase_df_ab.csv')
active_release_history = pd.read_csv('/Users/kaziamithasan/Desktop/Research/msr25/msr25-mining-challenge/phase_df_ac.csv')

In [2]:
active_release_history

,artifact_id,phase,start_date,end_date,duration (days)
0,aero.m-click:mcpdf,Active,2015-05-02 01:12:05,2015-05-03 21:07:03,1
1,aero.m-click:mcpdf,Dormant,2015-05-03 21:07:03,2023-01-25 22:10:44,2824
2,aero.m-click:mcpdf,Active,2023-01-25 22:10:44,2023-01-28 01:42:16,2
3,africa.shuwari.sbt:sbt-shuwari-core_2.12_1.0,Active,2023-08-01 19:26:50,2024-04-02 12:45:18,244
4,africa.shuwari.sbt:sbt-shuwari-header_2.12_1.0,Active,2023-08-01 19:26:51,2024-04-02 12:45:18,244
...,...,...,...,...,...
293949,zone.src.sheaf:web-sheaf,Active,2015-01-23 14:53:59,2015-01-23 14:53:59,0
293950,zone.src.sheaf:web-sheaf,Dormant,2015-01-23 14:53:59,2016-02-07 13:16:14,379
293951,zone.src.sheaf:web-sheaf,Active,2016-02-07 13:16:14,2017-12-13 16:03:48,675
293952,zone.src.sheaf:web-sheaf,Dormant,2017-12-13 16:03:48,2022-08-25 17:07:58,1716


In [3]:
# 

# Group by artifact_id and get the index of the last release for each project
last_release_indices = active_release_history.groupby('artifact_id').tail(1).index

# Update the phase label of the last release to 'Active_again'
active_release_history.loc[last_release_indices, 'phase'] = 'Active_again'

# Display the updated DataFrame
active_release_history


,artifact_id,phase,start_date,end_date,duration (days)
0,aero.m-click:mcpdf,Active,2015-05-02 01:12:05,2015-05-03 21:07:03,1
1,aero.m-click:mcpdf,Dormant,2015-05-03 21:07:03,2023-01-25 22:10:44,2824
2,aero.m-click:mcpdf,Active_again,2023-01-25 22:10:44,2023-01-28 01:42:16,2
3,africa.shuwari.sbt:sbt-shuwari-core_2.12_1.0,Active_again,2023-08-01 19:26:50,2024-04-02 12:45:18,244
4,africa.shuwari.sbt:sbt-shuwari-header_2.12_1.0,Active_again,2023-08-01 19:26:51,2024-04-02 12:45:18,244
...,...,...,...,...,...
293949,zone.src.sheaf:web-sheaf,Active,2015-01-23 14:53:59,2015-01-23 14:53:59,0
293950,zone.src.sheaf:web-sheaf,Dormant,2015-01-23 14:53:59,2016-02-07 13:16:14,379
293951,zone.src.sheaf:web-sheaf,Active,2016-02-07 13:16:14,2017-12-13 16:03:48,675
293952,zone.src.sheaf:web-sheaf,Dormant,2017-12-13 16:03:48,2022-08-25 17:07:58,1716


In [7]:
import pandas as pd

# Function to capture the full sequence of states with time intervals for each project
def capture_full_sequence(df):
    # Sort data by artifact_id and start_date for each project
    df = df.sort_values(by=['artifact_id', 'start_date'])
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    # Initialize a list to store the full sequence data
    sequence_data = []
    
    # Iterate over each project (grouped by artifact_id)
    for artifact_id, group in df.groupby('artifact_id'):
        project_sequence = []
        prev_end_date = None  # Track the previous end date for calculating intervals
        
        # Iterate over each state for the project
        for _, row in group.iterrows():
            current_state = row['phase']
            start_date = row['start_date']
            end_date = row['end_date']
            duration_days = row['duration (days)']
            
            # # Calculate the gap from the previous state, if any
            # if prev_end_date is not None:
            #     gap_between_states = start_date - prev_end_date
            # else:
            #     gap_between_states = None  # The first state has no previous gap
            
            # Append the current state and its associated data to the sequence
            project_sequence.append({
                'state': current_state,
                'start_date': start_date,
                'end_date': end_date,
                'duration (days)': duration_days
                #'gap_from_previous_state': gap_between_states
            })
            
            # Update the previous end date to the current end date
            prev_end_date = end_date
        
        # Append the entire sequence for the current project to the results
        sequence_data.append({
            'artifact_id': artifact_id,
            'full_sequence': project_sequence
        })
    
    # Convert the results into a DataFrame
    return pd.DataFrame(sequence_data)

# Apply the function to capture the full sequence for each project in the dataset
full_sequence_df_active = capture_full_sequence(active_release_history)

# Display the first few rows of the result
full_sequence_df_active

,artifact_id,full_sequence
0,aero.m-click:mcpdf,"[{'state': 'Active', 'start_date': 2015-05-02 01:12:05, 'end_date': 2015-05-03 21:07:03, 'duration (days)': 1}, {'state': 'Dormant', 'start_date': 2015-05-03 21:07:03, 'end_date': 2023-01-25 22:10:44, 'duration (days)': 2824}, {'state': 'Active_again', 'start_date': 2023-01-25 22:10:44, 'end_date': 2023-01-28 01:42:16, 'duration (days)': 2}]"
1,africa.shuwari.sbt:sbt-shuwari-core_2.12_1.0,"[{'state': 'Active_again', 'start_date': 2023-08-01 19:26:50, 'end_date': 2024-04-02 12:45:18, 'duration (days)': 244}]"
2,africa.shuwari.sbt:sbt-shuwari-header_2.12_1.0,"[{'state': 'Active_again', 'start_date': 2023-08-01 19:26:51, 'end_date': 2024-04-02 12:45:18, 'duration (days)': 244}]"
3,africa.shuwari.sbt:sbt-shuwari-js_2.12_1.0,"[{'state': 'Active_again', 'start_date': 2023-08-01 19:26:49, 'end_date': 2024-04-02 12:45:17, 'duration (days)': 244}]"
4,africa.shuwari.sbt:sbt-shuwari-mode_2.12_1.0,"[{'state': 'Active_again', 'start_date': 2023-08-01 19:26:51, 'end_date': 2024-04-02 12:45:19, 'duration (days)': 244}]"
...,...,...
196275,zone.src.sheaf:java-sheaf,"[{'state': 'Active', 'start_date': 2015-01-23 11:38:51, 'end_date': 2017-12-13 15:47:55, 'duration (days)': 1055}, {'state': 'Dormant', 'start_date': 2017-12-13 15:47:55, 'end_date': 2022-08-25 16:55:46, 'duration (days)': 1716}, {'state': 'Active_again', 'start_date': 2022-08-25 16:55:46, 'end_date': 2023-08-21 12:09:42, 'duration (days)': 360}]"
196276,zone.src.sheaf:logback-sheaf,"[{'state': 'Active', 'start_date': 2015-03-20 08:18:15, 'end_date': 2017-12-12 11:11:46, 'duration (days)': 998}, {'state': 'Dormant', 'start_date': 2017-12-12 11:11:46, 'end_date': 2022-08-25 11:09:24, 'duration (days)': 1716}, {'state': 'Active_again', 'start_date': 2022-08-25 11:09:24, 'end_date': 2023-08-21 10:56:45, 'duration (days)': 360}]"
196277,zone.src.sheaf:sheaf-deps-bom,"[{'state': 'Active', 'start_date': 2017-05-06 11:42:13, 'end_date': 2017-12-13 15:40:31, 'duration (days)': 221}, {'state': 'Dormant', 'start_date': 2017-12-13 15:40:31, 'end_date': 2022-08-25 16:05:27, 'duration (days)': 1716}, {'state': 'Active_again', 'start_date': 2022-08-25 16:05:27, 'end_date': 2023-08-21 11:56:48, 'duration (days)': 360}]"
196278,zone.src.sheaf:sheaf-parent,"[{'state': 'Active', 'start_date': 2015-01-23 10:37:20, 'end_date': 2017-12-12 11:44:37, 'duration (days)': 1054}, {'state': 'Dormant', 'start_date': 2017-12-12 11:44:37, 'end_date': 2022-08-25 15:53:20, 'duration (days)': 1717}, {'state': 'Active_again', 'start_date': 2022-08-25 15:53:20, 'end_date': 2023-08-21 09:29:38, 'duration (days)': 360}]"


In [63]:
import pandas as pd

# Function to capture the full sequence of states with time intervals for each project
def capture_full_sequence(df):
    # Sort data by artifact_id and start_date for each project
    df = df.sort_values(by=['artifact_id', 'start_date'])
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    # Initialize a list to store the full sequence data
    sequence_data = []
    
    # Iterate over each project (grouped by artifact_id)
    for artifact_id, group in df.groupby('artifact_id'):
        project_sequence = []
        prev_end_date = None  # Track the previous end date for calculating intervals
        
        # Iterate over each state for the project
        for _, row in group.iterrows():
            current_state = row['phase']
            start_date = row['start_date']
            end_date = row['end_date']
            duration_days = row['duration (days)']
            
            # # Calculate the gap from the previous state, if any
            # if prev_end_date is not None:
            #     gap_between_states = start_date - prev_end_date
            # else:
            #     gap_between_states = None  # The first state has no previous gap
            
            # Append the current state and its associated data to the sequence
            project_sequence.append({
                'state': current_state,
                'start_date': start_date,
                'end_date': end_date,
                'duration (days)': duration_days
                #'gap_from_previous_state': gap_between_states
            })
            
            # Update the previous end date to the current end date
            prev_end_date = end_date
        
        # Append the entire sequence for the current project to the results
        sequence_data.append({
            'artifact_id': artifact_id,
            'full_sequence': project_sequence
        })
    
    # Convert the results into a DataFrame
    return pd.DataFrame(sequence_data)

# Apply the function to capture the full sequence for each project in the dataset
full_sequence_df_ab = capture_full_sequence(abandoned_release_history)

# Display the first few rows of the result
full_sequence_df_ab

,artifact_id,full_sequence
0,academy.compose.companion:multi-fab,"[{'state': 'Active', 'start_date': 2021-01-05 21:01:38, 'end_date': 2021-01-14 06:40:25, 'duration (days)': 8.0}, {'state': 'Abandoned', 'start_date': 2021-01-14 06:40:25, 'end_date': NaT, 'duration (days)': nan}]"
1,ae.teletronics.ejabberd:EjabberdXMLRPCClient,"[{'state': 'Active', 'start_date': 2016-10-27 04:48:11, 'end_date': 2017-03-14 08:38:12, 'duration (days)': 138.0}, {'state': 'Abandoned', 'start_date': 2017-03-14 08:38:12, 'end_date': NaT, 'duration (days)': nan}]"
2,ae.teletronics.nlp:categorisation,"[{'state': 'Active', 'start_date': 2016-04-11 20:23:38, 'end_date': 2017-01-05 08:41:51, 'duration (days)': 268.0}, {'state': 'Abandoned', 'start_date': 2017-01-05 08:41:51, 'end_date': NaT, 'duration (days)': nan}]"
3,ae.teletronics.nlp:entityextraction,"[{'state': 'Active', 'start_date': 2016-06-15 06:49:12, 'end_date': 2016-09-30 12:52:02, 'duration (days)': 107.0}, {'state': 'Abandoned', 'start_date': 2016-09-30 12:52:02, 'end_date': NaT, 'duration (days)': nan}]"
4,ae.teletronics.nlp:language-detector,"[{'state': 'Active', 'start_date': 2016-02-16 10:54:43, 'end_date': 2016-04-26 06:41:12, 'duration (days)': 69.0}, {'state': 'Abandoned', 'start_date': 2016-04-26 06:41:12, 'end_date': NaT, 'duration (days)': nan}]"
...,...,...
206763,zone.refactor.spring:hateoas,"[{'state': 'Active', 'start_date': 2019-10-22 05:17:14, 'end_date': 2019-12-03 07:59:12, 'duration (days)': 42.0}, {'state': 'Abandoned', 'start_date': 2019-12-03 07:59:12, 'end_date': NaT, 'duration (days)': nan}]"
206764,zone.refactor.spring:validation,"[{'state': 'Active', 'start_date': 2019-10-26 09:03:16, 'end_date': 2019-11-26 20:53:27, 'duration (days)': 31.0}, {'state': 'Abandoned', 'start_date': 2019-11-26 20:53:27, 'end_date': NaT, 'duration (days)': nan}]"
206765,zone.stefan.dev:geocode,"[{'state': 'Active', 'start_date': 2021-01-08 15:30:37, 'end_date': 2021-01-08 17:44:45, 'duration (days)': 0.0}, {'state': 'Abandoned', 'start_date': 2021-01-08 17:44:45, 'end_date': NaT, 'duration (days)': nan}]"
206766,zone.wmj:user-agent-util,"[{'state': 'Active', 'start_date': 2021-09-10 17:26:29, 'end_date': 2022-06-16 07:45:05, 'duration (days)': 278.0}, {'state': 'Abandoned', 'start_date': 2022-06-16 07:45:05, 'end_date': NaT, 'duration (days)': nan}]"


In [58]:
from collections import Counter

# Assuming 'abandoned_full_sequence' DataFrame has a 'simplified_sequence' column

# Step 1: Extract all transitions from the simplified sequences
transitions = []

for _, row in full_sequence_df_ab.iterrows():
    sequence = row['full_sequence']
    
    # Capture consecutive transitions within the sequence
    for i in range(1, len(sequence)):
        from_state = sequence[i - 1]['state']
        to_state = sequence[i]['state']
        
        # Append each transition as a tuple
        transitions.append((from_state, to_state))

# Step 2: Count the occurrences of each transition
transition_counts = Counter(transitions)

# Step 3: Calculate total transitions from each state
total_transitions_from_state = Counter([from_state for from_state, _ in transitions])

# Step 4: Calculate transition probabilities
transition_probabilities = {}

for (from_state, to_state), count in transition_counts.items():
    # Calculate the probability of transition from 'from_state' to 'to_state'
    probability = (count / total_transitions_from_state[from_state]) * 100  # Convert to percentage
    transition_probabilities[(from_state, to_state)] = round(probability, 2)

# Step 5: Convert the results into a DataFrame for easy viewing
transition_df = pd.DataFrame(
    [(from_state, to_state, prob) for (from_state, to_state), prob in transition_probabilities.items()],
    columns=['From State', 'To State', 'Transition Probability (%)']
)

# Sort for better readability
transition_df.sort_values(by=['From State', 'To State'], inplace=True)
transition_df.reset_index(drop=True, inplace=True)

# Display the DataFrame
transition_df


,From State,To State,Transition Probability (%)
0,Active,Abandoned,82.89
1,Active,Dormant,17.11
2,Dormant,Abandoned,51.94
3,Dormant,Active,48.06


In [77]:
import pandas as pd
from collections import Counter

# Step 1: Extract all transitions from full_sequence
def extract_transitions(full_sequence_df):
    transitions = []
    for sequence in full_sequence_df['full_sequence']:
        for i in range(len(sequence) - 1):  # Pairwise transitions
            transitions.append((sequence[i]['state'], sequence[i+1]['state']))
    return transitions

# Extract transitions
transitions = extract_transitions(full_sequence_df_ab)

# Count occurrences of transitions
transition_counts = Counter(transitions)

# Get all unique states
unique_states = list(set(state for transition in transition_counts.keys() for state in transition))

# Step 2: Build Transition Matrix
# Create an empty transition matrix
transition_matrix = pd.DataFrame(0, index=unique_states, columns=unique_states, dtype=float)

# Populate the matrix with transition counts
for (from_state, to_state), count in transition_counts.items():
    transition_matrix.loc[from_state, to_state] += count

# Normalize rows to calculate probabilities
transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)

# Display the transition matrix
transition_matrix

,Dormant,Active,Abandoned
Dormant,0.000000,0.480642,0.519358
Active,0.171059,0.000000,0.828941
Abandoned,NaN,NaN,NaN


In [78]:
# Query a specific multi-step probability: Active → Dormant → Active
prob_active_dormant_active = (
    transition_matrix.loc["Active", "Dormant"] * 
    transition_matrix.loc["Dormant", "Active"]
)

print(f"Probability of 'Active → Dormant → Active': {prob_active_dormant_active}")


Probability of 'Active → Dormant → Active': 0.08221810902344079


In [80]:
# Step 1: Define a function to compute multi-step probabilities
import itertools

def compute_multi_step_probabilities(matrix, steps=3):
    states = list(matrix.index)  # Get all states
    paths = list(itertools.product(states, repeat=steps))  # All possible multi-step paths
    results = []
    
    for path in paths:
        prob = 1.0
        # Multiply probabilities for the given path
        for i in range(len(path) - 1):
            prob *= matrix.loc[path[i], path[i+1]]
        
        # Only include paths with non-zero probability
        if prob > 0:
            results.append({"Path": " → ".join(path), "Probability": prob})
    
    # Convert results to a DataFrame
    return pd.DataFrame(results).sort_values(by="Probability", ascending=False)

# Step 2: Compute probabilities for all 3-step transitions
multi_step_results = compute_multi_step_probabilities(transition_matrix, steps=3)
multi_step_results

,Path,Probability
1,Dormant → Active → Abandoned,0.398424
3,Active → Dormant → Abandoned,0.088841
0,Dormant → Active → Dormant,0.082218
2,Active → Dormant → Active,0.082218


In [26]:
# Prepare transactions from consecutive state transitions
transactions = []

for _, row in full_sequence_df_ab.iterrows():
    sequence = row['full_sequence']
    
    # Extract consecutive transitions within the sequence
    for i in range(1, len(sequence)):
        from_state = sequence[i - 1]['state']
        to_state = sequence[i]['state']
        
        # Include the pair as part of the transaction
        transactions.append([from_state, from_state, to_state])  # To capture Active, Active → Other
transactions

[['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Dormant'],
 ['Dormant', 'Dormant', 'Active'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Dormant'],
 ['Dormant', 'Dormant', 'Active'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned'],
 ['Active', 'Active', 'Abandoned']

In [36]:
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Encode transactions into a binary matrix
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Run FP-Growth
frequent_itemsets = fpgrowth(df_encoded, min_support=0.000001, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.000001)

# Display the rules
rules.sort_values(by='confidence', ascending=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Abandoned),(Active),0.783720,0.924136,0.707856,0.903201,0.977346,-0.016408,0.783720,-0.096799
1,(Active),(Abandoned),0.924136,0.783720,0.707856,0.765965,0.977346,-0.016408,0.924136,-0.234035
2,(Dormant),(Active),0.292144,0.924136,0.216280,0.740321,0.801095,-0.053700,0.292144,-0.259679
4,(Dormant),(Abandoned),0.292144,0.783720,0.075864,0.259679,0.331341,-0.153095,0.292144,-0.740321
3,(Active),(Dormant),0.924136,0.292144,0.216280,0.234035,0.801095,-0.053700,0.924136,-0.765965
5,(Abandoned),(Dormant),0.783720,0.292144,0.075864,0.096799,0.331341,-0.153095,0.783720,-0.903201


In [40]:
from prefixspan import PrefixSpan

# Run PrefixSpan on the sequences
ps = PrefixSpan(sequences)

# Find top-k frequent patterns
top_patterns = ps.topk(k=20, closed=True)  # k=10 for top 10 patterns, closed=True for closed patterns

# Display the results
for pattern in top_patterns:
    print(f"Support: {pattern[0]}, Sequence: {pattern[1]}")


Support: 196280, Sequence: ['Active_again']
Support: 47111, Sequence: ['Active', 'Active_again']
Support: 31817, Sequence: ['Active', 'Dormant', 'Active_again']
Support: 9465, Sequence: ['Active', 'Dormant', 'Active', 'Active_again']
Support: 6045, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active_again']
Support: 1854, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Active_again']
Support: 1158, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active_again']
Support: 141, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Active_again']
Support: 81, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active_again']
Support: 1, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active_again']


In [49]:
from prefixspan import PrefixSpan

# Extract sequences for PrefixSpan
sequences = [[state['state'] for state in seq] for seq in full_sequence_df_ab['full_sequence']]

# Run PrefixSpan for sequential patterns
ps = PrefixSpan(sequences)
patterns = ps.topk(10, closed=True)  # Get top 10 patterns

# Display patterns
for pattern in patterns:
    print(f"Support: {pattern[0]}, Sequence: {pattern[1]}")


Support: 206768, Sequence: ['Active', 'Abandoned']
Support: 34943, Sequence: ['Active', 'Dormant', 'Abandoned']
Support: 17141, Sequence: ['Active', 'Dormant', 'Active', 'Abandoned']
Support: 3334, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Abandoned']
Support: 1290, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Abandoned']
Support: 255, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Abandoned']
Support: 88, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Abandoned']
Support: 6, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Abandoned']
Support: 4, Sequence: ['Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Dormant', 'Active', 'Abandoned']


In [45]:
import pandas as pd

# Function to capture the full sequence of states with time intervals for each project
def capture_full_sequence(df):
    # Sort data by artifact_id and release_timestamp for each project
    df = df.sort_values(by=['artifact_id', 'release_timestamp'])
    
    # Initialize a list to store the full sequence data
    sequence_data = []
    
    # Iterate over each project (grouped by artifact_id)
    for artifact_id, group in df.groupby('artifact_id'):
        project_sequence = []
        prev_timestamp = None  # Track the previous timestamp for calculating intervals
        
        # Iterate over each release for the project
        for _, row in group.iterrows():
            current_state = row['label']
            current_timestamp = row['release_timestamp']
            
            # Calculate the time interval from the previous state if it exists
            if prev_timestamp is not None:
                time_interval = current_timestamp - prev_timestamp
            else:
                time_interval = None  # The first state has no previous time interval
            
            # Append the current state and its time interval to the sequence
            project_sequence.append({
                'state': current_state,
                'time_interval': time_interval
            })
            
            # Update the previous timestamp to the current one
            prev_timestamp = current_timestamp
        
        # Append the entire sequence for the current project to the results
        sequence_data.append({
            'artifact_id': artifact_id,
            'full_sequence': project_sequence
        })
    
    # Convert the results into a DataFrame
    return pd.DataFrame(sequence_data)

# Apply the function to capture the full sequence for each project in abandoned_release_history
abandoned_full_sequence = capture_full_sequence(abandoned_release_history)

# Display the first few rows of the result
abandoned_full_sequence.head()


,artifact_id,full_sequence
0,academy.compose.companion:multi-fab,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 1 days 23:42:44}, {'state': 'Abandoned', 'time_interval': 6 days 09:56:03}]"
1,ae.teletronics.ejabberd:EjabberdXMLRPCClient,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 136 days 03:41:19}, {'state': 'Active', 'time_interval': 1 days 00:18:22}, {'state': 'Abandoned', 'time_interval': 0 days 23:50:20}]"
2,ae.teletronics.nlp:categorisation,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 7 days 09:58:42}, {'state': 'Active', 'time_interval': 164 days 04:03:06}, {'state': 'Active', 'time_interval': 11 days 01:38:49}, {'state': 'Active', 'time_interval': 20 days 01:37:44}, {'state': 'Abandoned', 'time_interval': 65 days 18:59:52}]"
3,ae.teletronics.nlp:entityextraction,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 2 days 02:56:29}, {'state': 'Active', 'time_interval': 2 days 21:59:14}, {'state': 'Active', 'time_interval': 77 days 02:42:56}, {'state': 'Abandoned', 'time_interval': 25 days 02:24:11}]"
4,ae.teletronics.nlp:language-detector,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 6 days 00:12:22}, {'state': 'Active', 'time_interval': 15 days 21:02:22}, {'state': 'Active', 'time_interval': 0 days 01:52:42}, {'state': 'Active', 'time_interval': 46 days 19:22:09}, {'state': 'Abandoned', 'time_interval': 1 days 01:16:54}]"


In [46]:
import pandas as pd

# Function to capture the full sequence of states with time intervals for each project
def capture_full_sequence(df):
    # Sort data by artifact_id and release_timestamp for each project
    df = df.sort_values(by=['artifact_id', 'release_timestamp'])
    
    # Initialize a list to store the full sequence data
    sequence_data = []
    
    # Iterate over each project (grouped by artifact_id)
    for artifact_id, group in df.groupby('artifact_id'):
        project_sequence = []
        prev_timestamp = None  # Track the previous timestamp for calculating intervals
        
        # Iterate over each release for the project
        for _, row in group.iterrows():
            current_state = row['label']
            current_timestamp = row['release_timestamp']
            
            # Calculate the time interval from the previous state if it exists
            if prev_timestamp is not None:
                time_interval = current_timestamp - prev_timestamp
            else:
                time_interval = None  # The first state has no previous time interval
            
            # Append the current state and its time interval to the sequence
            project_sequence.append({
                'state': current_state,
                'time_interval': time_interval
            })
            
            # Update the previous timestamp to the current one
            prev_timestamp = current_timestamp
        
        # Append the entire sequence for the current project to the results
        sequence_data.append({
            'artifact_id': artifact_id,
            'full_sequence': project_sequence
        })
    
    # Convert the results into a DataFrame
    return pd.DataFrame(sequence_data)

# Apply the function to capture the full sequence for each project in abandoned_release_history
active_full_sequence = capture_full_sequence(active_release_history)

# Display the first few rows of the result
active_full_sequence.head()


,artifact_id,full_sequence
0,aero.m-click:mcpdf,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 1 days 19:54:58}, {'state': 'Dormant', 'time_interval': 366 days 13:44:51}, {'state': 'Dormant', 'time_interval': 2457 days 11:18:50}, {'state': 'Active', 'time_interval': 0 days 00:48:34}, {'state': 'Active', 'time_interval': 0 days 00:13:27}, {'state': 'Active', 'time_interval': 2 days 01:28:58}, {'state': 'Active', 'time_interval': 0 days 00:39:41}, {'state': 'Active_again', 'time_interval': 0 days 00:20:52}]"
1,africa.shuwari.sbt:sbt-shuwari-core_2.12_1.0,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 12 days 18:58:13}, {'state': 'Active', 'time_interval': 5 days 03:25:19}, {'state': 'Active', 'time_interval': 82 days 07:12:27}, {'state': 'Active', 'time_interval': 131 days 20:12:18}, {'state': 'Active_again', 'time_interval': 12 days 15:30:11}]"
2,africa.shuwari.sbt:sbt-shuwari-header_2.12_1.0,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 12 days 18:58:11}, {'state': 'Active', 'time_interval': 5 days 03:25:19}, {'state': 'Active', 'time_interval': 82 days 07:12:28}, {'state': 'Active', 'time_interval': 131 days 20:12:18}, {'state': 'Active_again', 'time_interval': 12 days 15:30:11}]"
3,africa.shuwari.sbt:sbt-shuwari-js_2.12_1.0,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 12 days 18:58:15}, {'state': 'Active', 'time_interval': 5 days 03:25:18}, {'state': 'Active', 'time_interval': 82 days 07:12:28}, {'state': 'Active', 'time_interval': 131 days 20:12:17}, {'state': 'Active_again', 'time_interval': 12 days 15:30:10}]"
4,africa.shuwari.sbt:sbt-shuwari-mode_2.12_1.0,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 12 days 18:58:11}, {'state': 'Active', 'time_interval': 5 days 03:25:19}, {'state': 'Active', 'time_interval': 82 days 07:12:28}, {'state': 'Active', 'time_interval': 131 days 20:12:18}, {'state': 'Active_again', 'time_interval': 12 days 15:30:12}]"


In [47]:
import pandas as pd

# Function to identify projects with increasing active intervals
def find_increasing_intervals(full_sequence_df):
    increasing_intervals_projects = []

    for _, row in full_sequence_df.iterrows():
        project_id = row['artifact_id']
        sequence = row['full_sequence']
        active_intervals = []

        # Extract intervals between consecutive Active states
        for i in range(1, len(sequence)):
            if sequence[i - 1]['state'] == 'Active' and sequence[i]['state'] == 'Active':
                time_interval = sequence[i - 1]['time_interval']
                
                # Skip if time_interval is None
                if time_interval is not None:
                    active_intervals.append(time_interval.days)  # Convert to days

        # Check if intervals show an increasing trend
        if len(active_intervals) > 1:
            increasing_trend = all(x < y for x, y in zip(active_intervals, active_intervals[1:]))
            if increasing_trend:
                increasing_intervals_projects.append({
                    'artifact_id': project_id,
                    'intervals': active_intervals
                })

    return pd.DataFrame(increasing_intervals_projects)

# Apply the function to the abandoned and active datasets
abandoned_increasing_intervals = find_increasing_intervals(abandoned_full_sequence)
active_increasing_intervals = find_increasing_intervals(active_full_sequence)

print("Abandoned Projects with Increasing Active Intervals:")
print(abandoned_increasing_intervals)

print("\nActive Projects with Increasing Active Intervals:")
print(active_increasing_intervals)


Abandoned Projects with Increasing Active Intervals:
                                          artifact_id      intervals
0                               africa.absa:inception       [0, 155]
1                           africa.absa:inception-api       [0, 155]
2                   africa.absa:inception-application       [0, 155]
3                         africa.absa:inception-cache       [0, 155]
4                         africa.absa:inception-codes       [0, 155]
...                                               ...            ...
9682                        xyz.rk0cc.willpub:jpubdev  [0, 1, 2, 11]
9683  xyz.upperlevel.spigot.anvilgui:spigot-anvil-gui        [3, 41]
9684       za.co.absa.spline:spline-integration-tests   [37, 67, 86]
9685            za.co.absa.spline:spline-test-commons   [37, 67, 86]
9686                        za.co.monadic:scopus_2.11   [0, 99, 201]

[9687 rows x 2 columns]

Active Projects with Increasing Active Intervals:
                                  artifact_

In [48]:
abandoned_increasing_intervals

,artifact_id,intervals
0,africa.absa:inception,"[0, 155]"
1,africa.absa:inception-api,"[0, 155]"
2,africa.absa:inception-application,"[0, 155]"
3,africa.absa:inception-cache,"[0, 155]"
4,africa.absa:inception-codes,"[0, 155]"
...,...,...
9682,xyz.rk0cc.willpub:jpubdev,"[0, 1, 2, 11]"
9683,xyz.upperlevel.spigot.anvilgui:spigot-anvil-gui,"[3, 41]"
9684,za.co.absa.spline:spline-integration-tests,"[37, 67, 86]"
9685,za.co.absa.spline:spline-test-commons,"[37, 67, 86]"


In [49]:
active_increasing_intervals

,artifact_id,intervals
0,ai.bleckwen:xgboost-predictor4j,"[0, 279]"
1,ai.djl.tensorflow:tensorflow-native-cpu,"[29, 167, 288, 293]"
2,ai.picovoice:cheetah-java,"[77, 88]"
3,ai.salmonbrain:ruleofthumb_3.0.0_2.12,"[2, 25, 46]"
4,ai.salmonbrain:ruleofthumb_3.1.2_2.12,"[2, 25, 46]"
...,...,...
6051,xyz.cofe:cbuffer,"[0, 29]"
6052,xyz.grasscutters:grasscutter,"[3, 8, 25, 219]"
6053,xyz.silencelurker:fast-dubbo,"[0, 3, 108]"
6054,xyz.vfhhu.androidlib:androidlib,"[152, 279]"


In [50]:
# Function to detect early signs of dormancy in projects
def detect_early_dormancy(full_sequence_df, early_threshold=3):  # Check within first 3 transitions
    early_dormancy_projects = []

    for _, row in full_sequence_df.iterrows():
        project_id = row['artifact_id']
        sequence = row['full_sequence']

        # Check if 'Dormant' state appears within the first few transitions
        early_dormancy = any(step['state'] == 'Dormant' for step in sequence[:early_threshold])
        if early_dormancy:
            early_dormancy_projects.append({
                'artifact_id': project_id,
                'early_sequence': sequence[:early_threshold]
            })

    return pd.DataFrame(early_dormancy_projects)

# Apply the function to the abandoned and active datasets
abandoned_early_dormancy = detect_early_dormancy(abandoned_full_sequence)
active_early_dormancy = detect_early_dormancy(active_full_sequence)

print("Abandoned Projects with Early Dormancy:")
print(abandoned_early_dormancy)

print("\nActive Projects with Early Dormancy:")
print(active_early_dormancy)


Abandoned Projects with Early Dormancy:
                                          artifact_id  \
0                    ae.teletronics.solr:solr-plugins   
1                      ai.x:play-json-extensions_2.11   
2                             am.ik.marked4j:marked4j   
3     at.bestsolution:at.bestsolution.maven.publisher   
4             at.molindo:wicketstuff-merged-resources   
...                                               ...   
8740                                    xyz.cofe:text   
8741                              xyz.luan:parent-pom   
8742                 xyz.ronella.gosu:gs-codenarc-ext   
8743                   xyz.tneciv:api-version-mapping   
8744          zone.dragon.dropwizard:dropwizard-async   

                                                                                                                                                         early_sequence  
0     [{'state': 'Active', 'time_interval': None}, {'state': 'Dormant', 'time_interval': 761 days 23:08:1

In [51]:
abandoned_early_dormancy

,artifact_id,early_sequence
0,ae.teletronics.solr:solr-plugins,"[{'state': 'Active', 'time_interval': None}, {'state': 'Dormant', 'time_interval': 761 days 23:08:19}, {'state': 'Abandoned', 'time_interval': 20 days 23:33:23}]"
1,ai.x:play-json-extensions_2.11,"[{'state': 'Active', 'time_interval': None}, {'state': 'Dormant', 'time_interval': 463 days 17:43:09}, {'state': 'Active', 'time_interval': 0 days 01:30:58}]"
2,am.ik.marked4j:marked4j,"[{'state': 'Active', 'time_interval': None}, {'state': 'Dormant', 'time_interval': 508 days 02:25:22}, {'state': 'Active', 'time_interval': 0 days 00:31:11}]"
3,at.bestsolution:at.bestsolution.maven.publisher,"[{'state': 'Active', 'time_interval': None}, {'state': 'Dormant', 'time_interval': 457 days 17:23:25}, {'state': 'Abandoned', 'time_interval': 0 days 08:07:08}]"
4,at.molindo:wicketstuff-merged-resources,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 15 days 01:54:22}, {'state': 'Dormant', 'time_interval': 956 days 00:43:31}]"
...,...,...
8740,xyz.cofe:text,"[{'state': 'Active', 'time_interval': None}, {'state': 'Dormant', 'time_interval': 987 days 09:11:49}, {'state': 'Dormant', 'time_interval': 493 days 00:52:46}]"
8741,xyz.luan:parent-pom,"[{'state': 'Active', 'time_interval': None}, {'state': 'Active', 'time_interval': 108 days 20:18:51}, {'state': 'Dormant', 'time_interval': 836 days 00:32:40}]"
8742,xyz.ronella.gosu:gs-codenarc-ext,"[{'state': 'Active', 'time_interval': None}, {'state': 'Dormant', 'time_interval': 390 days 23:09:26}, {'state': 'Active', 'time_interval': 246 days 00:44:36}]"
8743,xyz.tneciv:api-version-mapping,"[{'state': 'Active', 'time_interval': None}, {'state': 'Dormant', 'time_interval': 404 days 16:17:30}, {'state': 'Active', 'time_interval': 0 days 02:39:41}]"


In [67]:
import pandas as pd

# Function to detect early signs of dormancy in projects with different thresholds
def detect_early_dormancy(full_sequence_df, thresholds=[3, 5, 10,500000]):
    results = []

    for threshold in thresholds:
        early_dormancy_projects = 0

        for _, row in full_sequence_df.iterrows():
            sequence = row['full_sequence']
            # Check if 'Dormant' state appears within the first 'threshold' transitions
            early_dormancy = any(step['state'] == 'Dormant' for step in sequence[:threshold])
            if early_dormancy:
                early_dormancy_projects += 1

        # Calculate the proportion of projects with early dormancy for the current threshold
        early_dormancy_ratio = early_dormancy_projects / len(full_sequence_df)
        results.append({
            'Threshold': threshold,
            'Early Dormancy Projects': early_dormancy_projects,
            'Total Projects': len(full_sequence_df),
            'Early Dormancy Ratio': round(early_dormancy_ratio * 100, 2)
        })

    return pd.DataFrame(results)

# Apply the function to both abandoned and active project datasets
thresholds = [3, 5, 10,500000]  # Test with different thresholds
abandoned_early_dormancy_summary = detect_early_dormancy(abandoned_full_sequence, thresholds)
active_early_dormancy_summary = detect_early_dormancy(active_full_sequence, thresholds)

# Summarize and compare the results
print("Abandoned Projects - Early Dormancy Summary:")
print(abandoned_early_dormancy_summary)

print("\nActive Projects - Early Dormancy Summary:")
print(active_early_dormancy_summary)


Abandoned Projects - Early Dormancy Summary:
   Threshold  Early Dormancy Projects  Total Projects  Early Dormancy Ratio
0          3                     8745          206768                  4.23
1          5                    12120          206768                  5.86
2         10                    15383          206768                  7.44
3     500000                    19070          206768                  9.22

Active Projects - Early Dormancy Summary:
   Threshold  Early Dormancy Projects  Total Projects  Early Dormancy Ratio
0          3                    11802          196280                  6.01
1          5                    18105          196280                  9.22
2         10                    25023          196280                 12.75
3     500000                    34602          196280                 17.63


In [53]:
import pandas as pd

# Function to analyze frequency and position of dormancy phases (early, middle, late)
def analyze_dormancy_frequency_position(full_sequence_df, end_threshold=5):
    dormancy_analysis_data = []

    for _, row in full_sequence_df.iterrows():
        project_id = row['artifact_id']
        sequence = row['full_sequence']
        total_phases = len(sequence)
        
        # Determine phase boundaries
        early_end = total_phases // 3
        middle_end = 2 * (total_phases // 3)

        # Count dormancy phases in each section of the lifecycle
        early_dormancy_count = sum(1 for step in sequence[:early_end] if step['state'] == 'Dormant')
        middle_dormancy_count = sum(1 for step in sequence[early_end:middle_end] if step['state'] == 'Dormant')
        late_dormancy_count = sum(1 for step in sequence[middle_end:] if step['state'] == 'Dormant')
        
        # Total dormancy count
        dormancy_count = early_dormancy_count + middle_dormancy_count + late_dormancy_count
        
        # Calculate the proportion of dormancy phases in each section
        early_ratio = early_dormancy_count / dormancy_count if dormancy_count > 0 else 0
        middle_ratio = middle_dormancy_count / dormancy_count if dormancy_count > 0 else 0
        late_ratio = late_dormancy_count / dormancy_count if dormancy_count > 0 else 0

        # Append results for the current project
        dormancy_analysis_data.append({
            'artifact_id': project_id,
            'total_dormancy_count': dormancy_count,
            'early_dormancy_count': early_dormancy_count,
            'middle_dormancy_count': middle_dormancy_count,
            'late_dormancy_count': late_dormancy_count,
            'early_ratio': round(early_ratio, 2),
            'middle_ratio': round(middle_ratio, 2),
            'late_ratio': round(late_ratio, 2)
        })

    return pd.DataFrame(dormancy_analysis_data)

# Apply the function to both abandoned and active project datasets
abandoned_dormancy_analysis = analyze_dormancy_frequency_position(abandoned_full_sequence)
active_dormancy_analysis = analyze_dormancy_frequency_position(active_full_sequence)

# Summarize the results by averaging across projects for both groups
abandoned_summary = abandoned_dormancy_analysis[['total_dormancy_count', 'early_ratio', 'middle_ratio', 'late_ratio']].mean()
active_summary = active_dormancy_analysis[['total_dormancy_count', 'early_ratio', 'middle_ratio', 'late_ratio']].mean()

print("Abandoned Projects - Dormancy Analysis Summary:")
print(abandoned_summary)

print("\nActive Projects - Dormancy Analysis Summary:")
print(active_summary)


Abandoned Projects - Dormancy Analysis Summary:
total_dormancy_count    0.110380
early_ratio             0.010842
middle_ratio            0.034352
late_ratio              0.047032
dtype: float64

Active Projects - Dormancy Analysis Summary:
total_dormancy_count    0.244014
early_ratio             0.031016
middle_ratio            0.059919
late_ratio              0.085337
dtype: float64


In [54]:
abandoned_dormancy_analysis

,artifact_id,total_dormancy_count,early_dormancy_count,middle_dormancy_count,late_dormancy_count,early_ratio,middle_ratio,late_ratio
0,academy.compose.companion:multi-fab,0,0,0,0,0.0,0.0,0.0
1,ae.teletronics.ejabberd:EjabberdXMLRPCClient,0,0,0,0,0.0,0.0,0.0
2,ae.teletronics.nlp:categorisation,0,0,0,0,0.0,0.0,0.0
3,ae.teletronics.nlp:entityextraction,0,0,0,0,0.0,0.0,0.0
4,ae.teletronics.nlp:language-detector,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
206763,zone.refactor.spring:hateoas,0,0,0,0,0.0,0.0,0.0
206764,zone.refactor.spring:validation,0,0,0,0,0.0,0.0,0.0
206765,zone.stefan.dev:geocode,0,0,0,0,0.0,0.0,0.0
206766,zone.wmj:user-agent-util,0,0,0,0,0.0,0.0,0.0


In [55]:
active_dormancy_analysis

,artifact_id,total_dormancy_count,early_dormancy_count,middle_dormancy_count,late_dormancy_count,early_ratio,middle_ratio,late_ratio
0,aero.m-click:mcpdf,2,1,1,0,0.50,0.5,0.00
1,africa.shuwari.sbt:sbt-shuwari-core_2.12_1.0,0,0,0,0,0.00,0.0,0.00
2,africa.shuwari.sbt:sbt-shuwari-header_2.12_1.0,0,0,0,0,0.00,0.0,0.00
3,africa.shuwari.sbt:sbt-shuwari-js_2.12_1.0,0,0,0,0,0.00,0.0,0.00
4,africa.shuwari.sbt:sbt-shuwari-mode_2.12_1.0,0,0,0,0,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...
196275,zone.src.sheaf:java-sheaf,2,0,0,2,0.00,0.0,1.00
196276,zone.src.sheaf:logback-sheaf,1,0,0,1,0.00,0.0,1.00
196277,zone.src.sheaf:sheaf-deps-bom,2,0,0,2,0.00,0.0,1.00
196278,zone.src.sheaf:sheaf-parent,2,0,0,2,0.00,0.0,1.00


In [56]:
active_dormancy_analysis.describe()

,total_dormancy_count,early_dormancy_count,middle_dormancy_count,late_dormancy_count,early_ratio,middle_ratio,late_ratio
count,196280.000000,196280.000000,196280.000000,196280.000000,196280.000000,196280.000000,196280.000000
mean,0.244014,0.046536,0.081297,0.116181,0.031016,0.059919,0.085337
std,0.599276,0.248646,0.302222,0.375400,0.162571,0.223712,0.268113
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,4.000000,4.000000,4.000000,1.000000,1.000000,1.000000


In [57]:
abandoned_dormancy_analysis.describe()

,total_dormancy_count,early_dormancy_count,middle_dormancy_count,late_dormancy_count,early_ratio,middle_ratio,late_ratio
count,206768.000000,206768.000000,206768.000000,206768.000000,206768.000000,206768.000000,206768.000000
mean,0.110380,0.013469,0.040461,0.056450,0.010842,0.034352,0.047032
std,0.374311,0.121004,0.206084,0.252657,0.098904,0.176299,0.206849
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,3.000000,3.000000,4.000000,1.000000,1.000000,1.000000


In [59]:
import pandas as pd

# Function to calculate maximum number of dormant phases within each project
def max_dormant_phases(full_sequence_df):
    max_dormancy_data = []

    for _, row in full_sequence_df.iterrows():
        project_id = row['artifact_id']
        sequence = row['full_sequence']

        # Count the number of "Dormant" phases in the project
        dormant_count = sum(1 for step in sequence if step['state'] == 'Dormant')
        
        # Append the result for the current project
        max_dormancy_data.append({
            'artifact_id': project_id,
            'max_dormant_phases': dormant_count
        })

    # Convert results to DataFrame
    return pd.DataFrame(max_dormancy_data)

# Apply the function to both abandoned and active project datasets
abandoned_max_dormancy = max_dormant_phases(abandoned_full_sequence)
active_max_dormancy = max_dormant_phases(active_full_sequence)

# Display the results
print("Abandoned Projects - Max Dormant Phases:")
print(abandoned_max_dormancy)

print("\nActive Projects - Max Dormant Phases:")
print(active_max_dormancy)


Abandoned Projects - Max Dormant Phases:
                                         artifact_id  max_dormant_phases
0                academy.compose.companion:multi-fab                   0
1       ae.teletronics.ejabberd:EjabberdXMLRPCClient                   0
2                  ae.teletronics.nlp:categorisation                   0
3                ae.teletronics.nlp:entityextraction                   0
4               ae.teletronics.nlp:language-detector                   0
...                                              ...                 ...
206763                  zone.refactor.spring:hateoas                   0
206764               zone.refactor.spring:validation                   0
206765                       zone.stefan.dev:geocode                   0
206766                      zone.wmj:user-agent-util                   0
206767                         zw.co.paynow:java-sdk                   0

[206768 rows x 2 columns]

Active Projects - Max Dormant Phases:
                 

In [62]:
abandoned_max_dormancy.describe()

,max_dormant_phases
count,206768.000000
mean,0.110380
std,0.374311
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,5.000000


In [63]:
active_max_dormancy.describe()

,max_dormant_phases
count,196280.000000
mean,0.244014
std,0.599276
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,5.000000


In [64]:
import pandas as pd

# Function to analyze the number of releases and dormant phases per project
def analyze_releases_vs_dormancy(full_sequence_df):
    release_dormancy_data = []

    for _, row in full_sequence_df.iterrows():
        project_id = row['artifact_id']
        sequence = row['full_sequence']

        # Total releases (states) in the project
        total_releases = len(sequence)

        # Count the number of "Dormant" phases in the project
        dormant_count = sum(1 for step in sequence if step['state'] == 'Dormant')

        # Determine if there is at least one dormant phase
        has_dormant_phase = dormant_count > 0

        # Append data for the current project
        release_dormancy_data.append({
            'artifact_id': project_id,
            'total_releases': total_releases,
            'has_dormant_phase': has_dormant_phase,
            'dormant_count': dormant_count
        })

    # Convert to DataFrame
    return pd.DataFrame(release_dormancy_data)

# Apply the function to both abandoned and active datasets
abandoned_release_dormancy = analyze_releases_vs_dormancy(abandoned_full_sequence)
active_release_dormancy = analyze_releases_vs_dormancy(active_full_sequence)

# Summary analysis for abandoned and active projects
abandoned_summary = abandoned_release_dormancy.groupby('has_dormant_phase')['total_releases'].describe()
active_summary = active_release_dormancy.groupby('has_dormant_phase')['total_releases'].describe()

print("Abandoned Projects - Releases vs Dormancy Summary:")
print(abandoned_summary)

print("\nActive Projects - Releases vs Dormancy Summary:")
print(active_summary)


Abandoned Projects - Releases vs Dormancy Summary:
                      count       mean        std  min  25%  50%   75%     max
has_dormant_phase                                                             
False              187698.0  14.194563  40.955153  2.0  3.0  5.0  12.0  2173.0
True                19070.0  12.105454  25.172788  3.0  5.0  8.0  13.0  1219.0

Active Projects - Releases vs Dormancy Summary:
                      count       mean         std  min  25%   50%   75%  \
has_dormant_phase                                                          
False              161678.0  58.937808  152.267392  2.0  7.0  19.0  54.0   
True                34602.0  20.236981   35.297011  3.0  6.0  12.0  23.0   

                      max  
has_dormant_phase          
False              3819.0  
True               3079.0  


In [65]:
abandoned_summary

,count,mean,std,min,25%,50%,75%,max
has_dormant_phase,,,,,,,,
False,187698.0,14.194563,40.955153,2.0,3.0,5.0,12.0,2173.0
True,19070.0,12.105454,25.172788,3.0,5.0,8.0,13.0,1219.0


In [66]:
active_summary

,count,mean,std,min,25%,50%,75%,max
has_dormant_phase,,,,,,,,
False,161678.0,58.937808,152.267392,2.0,7.0,19.0,54.0,3819.0
True,34602.0,20.236981,35.297011,3.0,6.0,12.0,23.0,3079.0
